In [5]:
!pip install ipykernel>=4.1.0
!pip install pydeck
!pip install streamlit
!pip install pyngrok
import streamlit as st
import pandas as pd
import datetime as dt
import numpy as np

In [286]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:////content/drive/MyDrive/Colab_Notebooks/CryptoDB.db')
#engine = create_engine('sqlite:///CryptoDB.db')
path = "/content/drive/MyDrive/Colab_Notebooks/"

In [296]:
symbols = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',
            engine).name.to_list()
symbols

['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'ADAUSDT',
 'XRPUSDT',
 'ONTUSDT',
 'ZRXUSDT',
 'CELRUSDT',
 'MATICUSDT',
 'ONEUSDT',
 'FTMUSDT',
 'DUSKUSDT',
 'MTLUSDT',
 'BUSDUSDT',
 'IOTXUSDT',
 'EURUSDT',
 'BNTUSDT',
 'SOLUSDT',
 'ETHUPUSDT',
 'BLZUSDT',
 'DOTUSDT',
 'DIAUSDT',
 'SUNUSDT',
 'HNTUSDT',
 'NEARUSDT',
 'CFXUSDT',
 'SHIBUSDT',
 'PHAUSDT',
 'BONDUSDT',
 'DEXEUSDT',
 'REQUSDT',
 'DYDXUSDT',
 'GALAUSDT',
 'LAZIOUSDT',
 'CVXUSDT',
 'JOEUSDT',
 'GLMRUSDT',
 'API3USDT',
 'TRXUSDT',
 'CHRUSDT',
 'CRVUSDT',
 'KSMUSDT',
 'EGLDUSDT',
 'CTKUSDT',
 'ROSEUSDT',
 'RAMPUSDT',
 'EPSUSDT',
 'BARUSDT',
 'MINAUSDT',
 'FIDAUSDT',
 'ATOMUSDT',
 'HBARUSDT',
 'ZENUSDT',
 'ADAUPUSDT',
 'ADADOWNUSDT',
 'MANAUSDT',
 'LUNAUSDT',
 'AVAXUSDT',
 'AAVEUSDT',
 'UNFIUSDT',
 'ALICEUSDT',
 'ERNUSDT',
 'KLAYUSDT',
 'FARMUSDT',
 'MCUSDT',
 'BICOUSDT',
 'LTCUSDT',
 'TUSDUSDT',
 'LRCUSDT',
 'LINKUPUSDT',
 'LINKDOWNUSDT',
 'AXSUSDT',
 'CKBUSDT',
 'ARUSDT',
 'ANYUSDT',
 'IMXUSDT',
 'ETCUSDT',
 'XMRUSDT',
 'DASHUSDT',
 'A

In [288]:
st.title('Welcome to the live TA platform')
#streamlit %run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]
#DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [289]:
def applyTechnicals(df):
  df['SMA_7'] = df.c.rolling(7).mean() # binance's existing two indicators
  df['SMA_25'] = df.c.rolling(25).mean()
  df.dropna(inplace=True)

**#Uncommant ALL following '#' while Cyrpto.db is running by Streaming_SQL_for_Signal_Streamlit.ipynb in local directory, to implement updated streaming data**

In [325]:
def qry(symbol): #uncommant ALL # while Cyrpto.db is running by Streaming_SQL_for_Signal_Streamlit.ipynb in local directory, to implement updated streaming data
  #now = dt.datetime.utcnow()
  #before = now - dt.timedelta(minutes=30)
  qry_str = f"""SELECT E,c FROM '{symbol}'"""# WHERE E>='{before}'"""
  df = pd.read_sql(qry_str,engine)
  df.E = pd.to_datetime(df.E)
  df = df.set_index('E')
  df = df.resample('1s').last() #-- Or even resample("1ms")/('1min') depends on hardware --#
  df.dropna(inplace=True)     #-- Drop Missing data --#
  applyTechnicals(df)
  df['position'] = np.where(df['SMA_7']>df['SMA_25'], 1, 0)
  return df

In [326]:
qry('BTCUSDT')

,c,SMA_7,SMA_25,position
E,,,,
2022-01-24 07:30:30,35237.54,35239.588571,35244.8228,0
2022-01-24 07:30:31,35235.00,35238.782857,35244.6476,0
2022-01-24 07:30:32,35235.00,35237.977143,35244.6476,0
2022-01-24 07:30:33,35235.00,35237.402857,35244.5680,0
2022-01-24 07:30:34,35232.55,35235.957143,35244.3188,0
...,...,...,...,...
2022-01-24 08:08:46,34988.84,34987.400000,34987.6792,0
2022-01-24 08:08:47,34988.84,34987.880000,34987.8504,1
2022-01-24 08:08:48,34978.55,34986.891429,34987.6100,0


In [374]:
'''if df['SMA_7']>df['SMA_25']:
  print('SELL',qry('BTCUSDT').position.diff()[-4])
elif df['SMA_7']<df['SMA_25']:
  print('BUY')'''
test = qry('BTCUSDT')
test['position'] = np.where(test['SMA_7']<test['SMA_25'], test['position']+1, test['position'])
test['position'] = np.where(test['SMA_7']>test['SMA_25'], test['position']-1, test['position'])
test

,c,SMA_7,SMA_25,position
E,,,,
2022-01-24 07:30:30,35237.54,35239.588571,35244.8228,1
2022-01-24 07:30:31,35235.00,35238.782857,35244.6476,1
2022-01-24 07:30:32,35235.00,35237.977143,35244.6476,1
2022-01-24 07:30:33,35235.00,35237.402857,35244.5680,1
2022-01-24 07:30:34,35232.55,35235.957143,35244.3188,1
...,...,...,...,...
2022-01-24 08:08:46,34988.84,34987.400000,34987.6792,1
2022-01-24 08:08:47,34988.84,34987.880000,34987.8504,0
2022-01-24 08:08:48,34978.55,34986.891429,34987.6100,1


In [290]:
def checkCorssSignal():
  for symbol in symbols:
    if len(qry(symbol).position) > 1: # at least two rows
       if qry(symbol).position[-1] and qry(symbol).position.diff()[-1]: #most recent cross detect
          st.write(symbol)


In [299]:
st.button('Get live SMA cross', on_click = checkCorssSignal())
# Downloaded this file as raw Python file .py
# Named it to CheckSMA.py
# Put it in my current Folder

False

In [300]:
!streamlit run CheckSMA.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...

Error: Invalid value: File does not exist: CheckSMA.py
